In [1]:
from glob import glob
import numpy as np
from polyphys.manage import organizer
from polyphys.manage import utilizer 
from polyphys.manage.parser import HnsCyl, HnsCub
from polyphys.analyze import clusters
from polyphys.analyze import measurer


In [15]:
database = '/Users/amirhsi_mini/research_data/'
wholes = organizer.sort_filenames(glob(database+"N*ens1*.npy"),fmts=['npy'])
wholes

[('/Users/amirhsi_mini/research_data/N200kbmm2nh12ac2l25epshc1nc0ens1.ring-nucleoid-distMatTMonHnsPatch.npy',),
 ('/Users/amirhsi_mini/research_data/N200kbmm2nh12ac2l25epshc1nc2387ens1.ring-nucleoid-distMatTMonHnsPatch.npy',),
 ('/Users/amirhsi_mini/research_data/N200kbmm2nh12ac2l25epshc1nc3581ens1.ring-nucleoid-distMatTMonHnsPatch.npy',),
 ('/Users/amirhsi_mini/research_data/N200kbmm2nh12ac2l25epshc1nc4775ens1.ring-nucleoid-distMatTMonHnsPatch.npy',),
 ('/Users/amirhsi_mini/research_data/N200kbmm2nh12ac2l25epshc1nc5968ens1.ring-nucleoid-distMatTMonHnsPatch.npy',),
 ('/Users/amirhsi_mini/research_data/N200kbmm2nh12ac2l25epshc1nc7162ens1.ring-nucleoid-distMatTMonHnsPatch.npy',),
 ('/Users/amirhsi_mini/research_data/N200kbmm2nh12ac2l25epshc1nc8356ens1.ring-nucleoid-distMatTMonHnsPatch.npy',),
 ('/Users/amirhsi_mini/research_data/N200kbmm2nh12ac2l25epshc1nc9549ens1.ring-nucleoid-distMatTMonHnsPatch.npy',),
 ('/Users/amirhsi_mini/research_data/N200kbmm2nh12ac2l25epshc1nc10743ens1.ring-nucl

In [18]:
whole = wholes[7][0]
a = np.load(whole)

a.shape

(20001, 200, 24)

In [29]:
# Pre-allocate some space
sim_info = HnsCub(
        whole,
        'whole',
        'cubic',
        'nucleoid',
        'ring'
    )
lj_cut = 2**(1/6)
r_cutoff = np.round(
    0.5 * lj_cut * (sim_info.dmon + sim_info.dhns_patch), 3
    ) 
n_frame , n_mon, n_hpatch = a.shape
n_hcore = n_hpatch // 2
cont_hcore_hcore_t = np.zeros((n_frame, n_hcore, n_hcore))
clusters_t = np.zeros((n_frame, n_hcore, n_hcore))
# Process each time step
for idx in range(n_frame):
    d_contact_m_hpatch = clusters.find_direct_contacts(
            a[idx], r_cutoff, inclusive=False
            )
    single_patch_dir_contact =clusters.enforce_single_patch_dir_contact(d_contact_m_hpatch)
    cont_m_hpatch = clusters.generate_contact_matrix(single_patch_dir_contact)
    # Asymmetric unsquared monomer-H-NS-core matrix:
    cont_m_hcore = np.logical_or(cont_m_hpatch[:, ::2], cont_m_hpatch[:, 1::2]).astype(int)
    # Symmetric squared H-NS-core-H-NS-core matrix:
    cont_hcore_hcore = np.matmul(cont_m_hcore.T, cont_m_hcore)
    # Store results
    cont_hcore_hcore_t[idx] = cont_hcore_hcore
    #clusters_stat = clusters.count_foci_clusters(cont_hcore_hcore)
    #clusters_t[idx] = clusters_stat


/Users/amirhsi_mini/OneDrive - University of Waterloo/PhD Research/Jupyter/PolyPhys/polyphys/manage/parser.py:1974: UserWarning: It is assumed that 'nc' is the last attribute short-key in a lineage_name of types: 'ensemble', 'ensemble_long', 'whole', 'segment'.
  warnings.warn(convention_warning, UserWarning)


In [49]:
rows, cols = np.nonzero(cont_m_hpatch)
indices_tuples = [(row, col) for row, col in zip(rows, cols)]
print(indices_tuples)

[(3, 22), (7, 7), (11, 16), (18, 6), (18, 23), (50, 3), (53, 21), (60, 14), (77, 19), (78, 15), (80, 2), (87, 18), (101, 12), (104, 13), (131, 9), (140, 4), (152, 11), (155, 5), (167, 10), (188, 8), (194, 20)]


In [50]:
cont_m_hpatch
rows, cols = np.nonzero(cont_m_hcore)
indices_tuples = [(row, col) for row, col in zip(rows, cols)]
print(indices_tuples)

[(3, 11), (7, 3), (11, 8), (18, 3), (18, 11), (50, 1), (53, 10), (60, 7), (77, 9), (78, 7), (80, 1), (87, 9), (101, 6), (104, 6), (131, 4), (140, 2), (152, 5), (155, 2), (167, 5), (188, 4), (194, 10)]


In [41]:
cont_m_hcore.sum()

21